# Satellite Imagery query, parse and render
The following notebook does a few things, it explores, how to query for images with the Google Earth Engine (GEE) Python API, filter using date ranges and metadata and adding image tiles to render from GEE.

In [2]:
import bqplot
import datetime
import dateutil.parser
import ee
import ipywidgets
import ipyleaflet
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets

# Configure the pretty printing output & initialize earthengine.
pp = pprint.PrettyPrinter(depth=4)
ee.Initialize()

In [3]:
# Function to get sizes in Human readable format
suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
def humansize(nbytes):
    i = 0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

*While single images are great to do quick analytics, the true power of the Earth Engine environment comes with the possibility of looking at really large and heavy image collections. In GEE environment image collections have their own characteristic setup and are often composed of multiple single images. They often have the similar band structure and generally share a similar metadata structure for filtering and querying.*

In this step we query an image collection and get the size and numbe of items in a collection. 

In [4]:
#Public Image Collections
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2018-08-01','2018-12-01').filterMetadata('CLOUD_COVER','less_than',2)

# Get collection size
print('Total number of LS-8 assets with filters: '+str(l8sr.size().getInfo()))
print('\n'+'Total size of LS-8 collection : '+str(humansize(l8sr.reduceColumns(ee.Reducer.sum(), ['system:asset_size']).getInfo()['sum'])))

Total number of LS-8 assets with filters: 10681

Total size of LS-8 collection : 5.68 TB


In [7]:
# Get sample image from collection
sample_image = ee.Image(l8sr.first())
pp.pprint(sample_image.getInfo())
band_names_original = sample_image.bandNames()
band_names_original.getInfo()

{'bands': [{'crs': 'EPSG:32628',
            'crs_transform': [30.0, 0.0, 341385.0, 0.0, -30.0, 8807715.0],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [9161, 9161],
            'id': 'B1'},
           {'crs': 'EPSG:32628',
            'crs_transform': [30.0, 0.0, 341385.0, 0.0, -30.0, 8807715.0],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensions': [9161, 9161],
            'id': 'B2'},
           {'crs': 'EPSG:32628',
            'crs_transform': [30.0, 0.0, 341385.0, 0.0, -30.0, 8807715.0],
            'data_type': {'max': 32767,
                          'min': -32768,
                          'precision': 'int',
                          'type': 'PixelType'},
            'dimensio

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

In [6]:
# Function to get tilelayer url from earthengine server
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

# Calculate a Spectral Index (NDVI)

The normalized difference vegetation index ($NDVI$) is a band ratio that is related to the amount of green vegetation. This formula is for the four band PlanetScope Surface Reflectance imagery.

\begin{equation*}
NDVI = \frac{NIR — RED}{NIR + RED} = \frac{Band4 — Band3}{Band4 + Band3}
\end{equation*}

where $NIR$ is the near infrared band and $RED$ is red band.

### Compare image with greenery

In [8]:
#Generate NDVI for Landsat SR
img = l8sr.median()
red = img.select(['B2']).rename('red')
nir = img.select(['B5']).rename('nir')
ndvi = (nir.subtract(red)).divide(nir.add(red)).rename('ndvi')

In [10]:
#Create a slider widget to add both Landsat-8 imagery without & w/o NDVI 
# Godavari - 19.0056509,78.164871
# KBR park - 17.4216604,78.4179574, 15.25
# Some US area - 37.4112,-122.0634
map2 = ipyleaflet.Map(
    center=(19.0056509,78.164871), zoom=12,
    layout={'height':'500px'},
)
l8sr_tile_url=GetTileLayerUrl(l8sr.median().visualize(min=600, max=4000, bands=['B4', 'B3', 'B2']))
ndvi_tile_url = GetTileLayerUrl(ndvi.visualize(bands=['ndvi'], min=-0.2, max=0.6, palette='grey,yellow,green'))  #Landsat 8 SR
left = ipyleaflet.TileLayer(url=l8sr_tile_url)
right=ipyleaflet.TileLayer(url=ndvi_tile_url)
control = ipyleaflet.SplitMapControl(left_layer=left, right_layer=right)
map2.add_control(control)
map2

Map(center=[19.0056509, 78.164871], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## Year wise comparison

### Create the image collection object


In [12]:
#Public Image Collections
l8sr_2014 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2014-08-01','2014-12-01').filterMetadata('CLOUD_COVER','less_than',2)
l8sr_2019 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2018-08-01','2018-12-01').filterMetadata('CLOUD_COVER','less_than',2)


In [13]:
#Generate NDVI for PlanetScope SR
img = l8sr_2014.median()
red = img.select(['B2']).rename('red')
nir = img.select(['B5']).rename('nir')
ndvi_2014 = (nir.subtract(red)).divide(nir.add(red)).rename('ndvi')

#Generate NDVI for PlanetScope SR
img = l8sr_2019.median()
red = img.select(['B2']).rename('red')
nir = img.select(['B5']).rename('nir')
ndvi_2019 = (nir.subtract(red)).divide(nir.add(red)).rename('ndvi')

In [15]:
#Create a slider widget to add both Landsat 8 and PlanetScope 4B SR imagery
# Godavari - 19.0056509,78.164871
# KBR park - 17.4216604,78.4179574, 15.25
# Some US area - 37.4112,-122.0634
# Mindspace - 17.4287132,78.3905353
map3 = ipyleaflet.Map(
    center=(19.2154792,72.9995944), zoom=12,
    layout={'height':'500px'},
)
# Uncomment for image comparison.
# l8sr2014_tile_url=GetTileLayerUrl(l8sr_2014.median().visualize(min=600, max=4000, bands=['B4', 'B3', 'B2']))
# l8sr2019_tile_url = GetTileLayerUrl(l8sr_2019.median().visualize(min=600, max=4000, bands=['B4', 'B3', 'B2']))  #Landsat 8 SR
ndvi2014_tile_url = GetTileLayerUrl(ndvi_2014.visualize(bands=['ndvi'], min=-0.2, max=0.6, palette='grey,yellow,green'))  #Landsat 8 SR
ndvi2019_tile_url = GetTileLayerUrl(ndvi_2019.visualize(bands=['ndvi'], min=-0.2, max=0.6, palette='grey,yellow,green'))  #Landsat 8 SR
# left = ipyleaflet.TileLayer(url=l8sr1985_tile_url)
# right=ipyleaflet.TileLayer(url=l8sr2019_tile_url)
left = ipyleaflet.TileLayer(url=ndvi2014_tile_url)
right=ipyleaflet.TileLayer(url=ndvi2019_tile_url)
control = ipyleaflet.SplitMapControl(left_layer=left, right_layer=right)
map3.add_control(control)
map3

Map(center=[19.2154792, 72.9995944], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

## Summary

We learned how to 
* Query an imagecollection using the Earth Engine API
* Get the count and size of a collection
* Adding a public dataset such as Landsat
* Generate a Normalized Difference Vegetation Index (NDVI) for Landsat 8 Surface Reflectance image